<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_7_Reducers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part from the free course [EEwPython](https://github.com/csaybar/EEwPython); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html).

<!--NAVIGATION-->
 < [Map and filter](1.6_Map_filter.ipynb) | [Contents](index.ipynb) |  [Iterations](1.8_Iterations.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.7_Reducers.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [0]:
#@title # Reducers



from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/LzxQH0Ze0iI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

In [0]:
#@title Function to visualize thumbnail URL
from IPython.display import Image
from IPython.display import HTML

def display_image(url,title="",CSS_row=False):
  img_link = "<center> <h1>%s</h1> <img title='%s' src= '%s'></center>" % (title,title,url)
  return HTML(img_link)

### Connecting GEE, Colab and Google Drive

- **Colab & Google Drive synchronization**

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

- **Colab & Earth Engine synchronization**

In [0]:
!pip install earthengine-api #Firstly we need the earth-engine API

In [0]:
!earthengine authenticate 

In [0]:
import ee
ee.Initialize()

### 1. Reducer overview

Reducers are the way to aggregate data over time, space, bands, arrays and other data structures in Earth Engine. The `ee.Reducer` class specifies how data is aggregated. The reducers in this class can specify a simple statistic to use for the aggregation (e.g. minimum, maximum, mean, median, standard deviation, etc.), or a more complex summary of the input data (e.g. histogram, linear regression, list). Reductions may occur over time (`imageCollection.reduce()`), space (`image.reduceRegion()`, `image.reduceNeighborhood()`), bands (`image.reduce()`), or the attribute space of a `FeatureCollection` (`featureCollection.reduceColumns()` or `FeatureCollection` methods that start with `aggregate_`).


####  Reducers have inputs and outputs

Reducers take an input dataset and produce a single output. When a single input reducer is applied to a multi-band image, Earth Engine automatically replicates the reducer and applies it separately to each band. As a result, the output image has the same number of bands as the input image; each band in the output is the reduction of pixels from the corresponding band in the input data. Some reducers take tuples of input datasets. These reducers will not be automatically replicated for each band. For example, `ee.Reducer.LinearRegression()` takes multiple predictor datasets (representing independent variables in the regression) in a particular order (see [Regression reducers](https://developers.google.com/earth-engine/reducers_regression)).

Some reducers produce multiple outputs, for example `ee.Reducer.minMax()`, `ee.Reducer.histogram()` or `ee.Reducer.toList()`. For example:





In [0]:
# Load and filter the Sentinel-2 image collection.
collection = ee.ImageCollection('COPERNICUS/S2')\
               .filterDate('2016-01-01', '2016-12-31')\
               .filterBounds(ee.Geometry.Point([-81.31, 29.90]))

# Reduce the collection.
extrema = collection.reduce(ee.Reducer.minMax())
extrema.getInfo()

This will produce an output with twice the number of bands of the inputs, where band names in the output have ‘\_min’ or ‘\_max’ appended to the band name.

The output type should match the computation. For example, a reducer applied to an `ImageCollection` has an `Image` output. Because the output is interpreted as a pixel value, you must use reducers with a numeric output to reduce an `ImageCollection` (reducers like `toList()` or `histogram()` won’t work).

### 2. ImageCollection Reductions

Consider the example of needing to take the median over a time series of images represented by an `ImageCollection`. To reduce an `ImageCollection`, use `imageCollection.reduce()`. This reduces the collection of images to an individual image as illustrated in Figure 1. Specifically, the output is computed pixel-wise, such that each pixel in the output is composed of the median value of all the images in the collection at that location. To get other statistics, such as mean, sum, variance, an arbitrary percentile, etc., the appropriate reducer should be selected and applied. For basic statistics like min, max, mean, etc., ImageCollection has shortcut methods like `min()`, `max()`, `mean()`, etc. They function in exactly the same way as calling `reduce()`, except the resultant band names will not have the name of the reducer appended.

<figure>
<center>
<img src="https://developers.google.com/earth-engine/images/Reduce_ImageCollection.png">
<figcaption>
  Figure 1. Illustration of an ee.Reducer applied to an ImageCollection.
</figcaption>
</center>
</figure>


For an example of reducing an ImageCollection, consider a collection of Landsat 5 images, filtered by path and row. The following code uses `reduce()` to reduce the collection to one `Image` (here a median reducer is used simply for illustrative purposes):

In [0]:
# Load an image collection, filtered so it's not too much data.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
               .filterDate('2008-01-01', '2008-12-31')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.median())

# The output is an Image.  Visualize!
vis_param = {'bands': ['B4_median', 'B3_median', 'B2_median'], 'gamma': 1.6}
area= ee.Geometry.Rectangle(-122.3355-1.2, 37.7924-1.2, -122.3355+1.2, 37.7924+1.2)
display_image(median.clip(area).getThumbUrl(vis_param),'Landsat-5')

### 3. Image Reductions

To reduce an `Image`, use `image.reduce()`. Reducing an image functions in an analogous way to `imageCollection.reduce()`, except the bands of the image are input to the reducer rather than the images in the collection. The output is also an image with number of bands equal to number of reducer outputs. For example:



In [0]:
# Load an image and select some bands of interest.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')\
          .select(['B4', 'B3', 'B2'])

# Reduce the image to get a one-band maximum value image.
maxValue = image.reduce(ee.Reducer.max())

# Display the result.
display_image(maxValue.getThumbURL({'max': 13000}))

### 4. Statistics of an Image Region

Suppose there is need to calculate statistics over a region (or regions) of an `ee.Image`. To get statistics of pixel values in an image region, use `image.reduceRegion()`. This reduces all the pixels in the region(s) to a statistic or other compact representation of the pixel data in the region (e.g. histogram). The region is represented as a `Geometry`, which might be a polygon, containing many pixels, or it might be a single point, in which case there will only be one pixel in the region. In either case, as illustrated in Figure 2, the output is a statistic derived from the pixels in the region.

<figure>
<center>
<img src="https://developers.google.com/earth-engine/images/Reduce_region_diagram.png">
<figcaption>
  Figure 2.  An illustration of an ee.Reducer applied to an image and a region.
</figcaption>
</center>
</figure>


For an example of getting pixel statistics in a region of an image using reduceRegion(), consider finding the mean spectral values of a 5-year Landsat composite within the boundaries of the Sierra Nevada Coniferous Forest:

In [0]:
# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Load an input region: Sierra Nevada mixed conifer forest.
region = ee.Feature(ee.FeatureCollection(
  'ft:1Ec8IWsP8asxN-ywSqgXWMuBaxI6pPaeh6hC64lA')
  .filter(ee.Filter.eq('G200_REGIO', 'Sierra Nevada Coniferous Forests'))
  .first())

# Reduce the region. The region parameter is the Feature geometry.
meanDictionary = image.reduceRegion(reducer = ee.Reducer.mean(),
                                    geometry = region.geometry(),
                                    scale = 30,
                                    maxPixels = 1e9)

# The result is a Dictionary.  Print it.
meanDictionary.getInfo()

Note that in this example the reduction is specified by providing the reducer (`ee.Reducer.mean()`), the geometry (`region.geometry()`), the scale (30 meters) and `maxPixels` for the maximum number of pixels to input to the reducer. A scale should always be specified in `reduceRegion()` calls. This is because in complex processing flows, which may involve data from different sources with different scales, the scale of the output will not be unambiguously determined from the inputs. In that case, the scale defaults to 1 degree, which generally produces unsatisfactory results. See this [page](https://developers.google.com/earth-engine/scale) for more information about how Earth Engine handles scale.



### 5. Statistics of Image Regions

To get image statistics in multiple regions stored in a `FeatureCollection`, you can use `image.reduceRegions()` to reduce multiple regions at once. The input to `reduceRegions()` is an `Image` and a `FeatureCollection`. The output is another `FeatureCollection` with the `reduceRegions()` output set as properties on each `Feature`. In this example, means of the Landsat 7 annual composite bands in each feature geometry will be added as properties to the input features:



In [0]:
# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Load a FeatureCollection of counties in Maine.
maineCounties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')\
                  .filter(ee.Filter.eq('StateName', 'Maine'))

# Add reducer output to the Features in the collection.
maineMeansFeatures = image.reduceRegions(collection = maineCounties,
                                         reducer = ee.Reducer.mean(),
                                         scale = 30)

# Print the first feature, to illustrate the result.
ee.Feature(maineMeansFeatures.first()).select(image.bandNames()).getInfo()

Observe that new properties, keyed by band name, have been added to the `FeatureCollection` to store the mean of the composite in each `Feature` geometry. 

### 6. Statistics of Image Neighborhoods

Rather than specifying a region over which to perform a reduction, it is also possible to specify a neighborhood in which to apply a reducer. To reduce image neighborhoods, use `image.reduceNeighborhood()`. In this case, the reduction will occur in a sliding window over the input image, with the window size and shape specified by an `ee.Kernel`. The output of `reduceNeighborhood()` will be another image, with each pixel value representing the output of the reduction in a neighborhood around that pixel in the input image. Figure 3 illustrates this type of reduction.

<figure>
<center>
<img src="https://developers.google.com/earth-engine/images/Reduce_Neighborhood.png">
<figcaption>
  Figure 3.  Illustration of reduceNeighborhood(), where the reducer is applied in a kernel.
</figcaption>
</center>
</figure>

For example, consider using National Agriculture Imagery Program (NAIP) imagery to quantify landscape differences resulting from logging in the California redwood forests. Specifically, use standard deviation (SD) in a neighborhood to represent the difference in texture between the logged area and the protected area. For example, to get texture of a NAIP Normalized Difference Vegetation Index (NDVI) image, use reduceNeighborhood() to compute SD in a neighborhood defined by a kernel:



In [0]:
# Define a region in the redwood forest.
redwoods = ee.Geometry.Rectangle(-124.0665, 41.0739, -123.934, 41.2029);

# Load input NAIP imagery and build a mosaic.
naipCollection = ee.ImageCollection('USDA/NAIP/DOQQ')\
                   .filterBounds(redwoods)\
                   .filterDate('2012-01-01', '2012-12-31')
naip = naipCollection.mosaic()

# Compute NDVI from the NAIP imagery.
naipNDVI = naip.normalizedDifference(['N', 'R'])

# Compute standard deviation (SD) as texture of the NDVI.
texture = naipNDVI.reduceNeighborhood(reducer = ee.Reducer.stdDev(),
                                      kernel = ee.Kernel.circle(7))

# Display the results.
img1 = display_image(naip.clip(redwoods).getThumbURL(),"NAIP imagery of the Northern California coast")
img2 = display_image(texture.clip(redwoods).getThumbURL({'min': 0, 'max': 0.3}),"NDVI - reduceNeighborhood()")
display(img1,img2)

<!--NAVIGATION-->
 < [Map and filter](1.6_Map_filter.ipynb) | [Contents](index.ipynb) |  [Iterations](1.8_Iterations.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.7_Reducers.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>